In [1]:
import pandas as pd
from RoutePlanner.CellGrid import CellGrid

#read in Ice  & Currents Points
icePoints     = pd.read_json('../resources/icepoints2019-02-08.json')
icePoints     = pd.DataFrame.from_records(icePoints.icepoints)
currentPoints = pd.read_json('../resources/currentPoints.json')
currentPoints = pd.DataFrame.from_records(currentPoints.currentPoints)

longMin    = 260
longMax    = 360
latMin     = -75
latMax     = -40
cellWidth  = 5
cellHeight = 2.5

cellGrid = CellGrid(longMin, longMax, latMin, latMax, cellWidth, cellHeight)
cellGrid.addIcePoints(icePoints)
cellGrid.addCurrentPoints(currentPoints)

from RoutePlanner.Optimisation import TravelTime
OptInfo = {}
OptInfo['WayPoints']            = pd.read_csv('../resources/WayPoints_TestExample2.csv') 
OptInfo['MaxIceExtent']         = 0.8
OptInfo['VehicleInfo']          = {}
OptInfo['VehicleInfo']['Speed'] = 26.3 #Units km/hr

TT = TravelTime(cellGrid,OptInfo)
TT.Dijkstra(verbrose=True)

# Loading Ground Truth & Comparing
GT = pd.read_csv('../resources/TestExample_GroundTruth2.csv')
GT['Long'] = GT['Long']+360
ax = TT.PlotPaths(waypoints=['Falklands'])
ax.plot(GT['Long'],GT['Lat'],'r')

In [ ]:
from RoutePlanner.Function import NewtonianCurve

In [ ]:
import numpy as np
Path = TT.Paths[1]


startPoint = Path['Path']['FullPath'][0,:][None,:]
endPoint   = Path['Path']['FullPath'][-1,:][None,:]

OrgcrossingPoints = np.concatenate([startPoint,
                                 Path['Path']['CrossingPoints'],
                                 endPoint])

Points = OrgcrossingPoints.copy()
iter=0
while iter < 100:
    for id in range(Points.shape[0]-2):
        Sp  = tuple(Points[id,:])
        Cp  = tuple(Points[id+1,:])
        Np  = tuple(Points[id+2,:])
        nc = NewtonianCurve(TT.Mesh,Sp,Cp,Np,TT.OptInfo['VehicleInfo']['Speed'],debugging=1,maxiter=500)
        TravelTime, CrossingPoint = nc.value()

        if id == 0:
            newPoints = CrossingPoint
        else:
            newPoints = np.concatenate([newPoints,CrossingPoint])
    Points = np.concatenate([startPoint,newPoints,endPoint])
    iter+=1



In [ ]:
ax = cellGrid.plot(return_ax=True)

crossingPoints = np.concatenate([Path['Path']['FullPath'][0,:][None,:],
                                 Path['Path']['CrossingPoints'],
                                 Path['Path']['FullPath'][-1,:][None,:]])

ax.scatter(Points[:,0],Points[:,1])
ax.plot(Points[:,0],Points[:,1])
ax.plot(GT['Long'],GT['Lat'],'r')
ax.set_xlim([295,325])
ax.set_ylim([-60,-50])